In [2]:
conda install scipy

Retrieving notices: ...working... done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=25.5.1



## Package Plan ##

  environment location: /Users/seoda-eun/anaconda3/envs/mkl_env

  added / updated specs:
    - scipy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scipy-1.11.4               |  py311h224febf_0        22.3 MB
    ------------------------------------------------------------
                                           Total:        22.3 MB

The following NEW packages will be INSTALLED:

  scipy              pkgs/main/osx-64::scipy-1.11.4-py311h224febf_0 



                                                                

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from scipy.linalg.blas import zhemm
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
from scipy.interpolate import CubicSpline
from scipy.integrate import solve_ivp
from scipy.linalg import cho_factor, cho_solve, lu_factor, lu_solve
import time

m_high = 1
m_low = -4
mdim = m_high - m_low + 1
ndim = 1
matrix_size = mdim * ndim  # 'size' 변수명 변경
"""
... (주석) ...
- F_bar, K_bar, G 행렬과 q 프로파일을 바이너리 파일에서 읽어 보간.
- 최종 F, K 행렬은 히트맵 플롯 시점에 필요한 샘플에 대해서만 즉석에서 계산하여 성능 저하를 방지.
"""
import numpy as np
import struct
import os
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

# ─────────────────────────────────────────────────────────────
# 1 & 2. 파일 읽기 관련 함수 (이전과 동일)
# ─────────────────────────────────────────────────────────────

def get_matrix_map(m, kind, n_blocks, mband_k=None, path=None):
    if kind in ('F', 'G'):
        for mb_test in range(m):
            count = sum(len(range(j, min(m, j + mb_test + 1))) for j in range(m))
            if count == n_blocks:
                mband = mb_test
                print(f"[{os.path.basename(path)}] mband = {mband} (으)로 추정 (F/G).")
                d = {k+1: (i,j) for k, (j,i) in enumerate(((j,i) for j in range(m) for i in range(j, min(m, j+mband+1))))}
                return d
        raise ValueError(f"F/G 행렬의 mband를 추정할 수 없습니다. (m={m}, n_blocks={n_blocks})")
    elif kind == 'K':
        if mband_k is None: raise ValueError("K 행렬을 처리하려면 mband_k 값을 지정해야 합니다.")
        print(f"[{os.path.basename(path)}] mband = {mband_k} (지정된 값) 사용 (K).")
        valid_element_count = sum(len(range(max(0, j-mband_k), min(m, j+mband_k+1))) for j in range(m))
        if n_blocks > valid_element_count:
            print(f"  > 경고: 파일에 기록된 블록 수({n_blocks})가 실제 유효한 원소 수({valid_element_count})보다 많습니다.")
        d = {}
        k = 0
        for j in range(m):
            for i in range(max(0, j - mband_k), min(m, j + mband_k + 1)):
                k += 1
                d[k] = (i, j)
        return d
    else: raise ValueError("kind는 'F', 'G', 'K' 중 하나여야 합니다.")

def read_bin(path, m, kind, mband_k=None):
    all_records = []
    with open(path, 'rb') as f:
        while True:
            header_bytes = f.read(4)
            if not header_bytes: break
            record_len = struct.unpack('<i', header_bytes)[0]
            if record_len > 0:
                all_records.append(np.frombuffer(f.read(record_len), dtype=np.float32))
            else:
                all_records.append(None)
            f.read(4)
    element_blocks = []
    current_block = []
    for rec in all_records:
        if rec is not None: current_block.append(rec)
        else:
            if current_block: element_blocks.append(np.array(current_block))
            current_block = []
    if not element_blocks: raise ValueError(f"파일에서 데이터 블록을 찾지 못했습니다: {path}")

    n_blocks, n_psi = len(element_blocks), element_blocks[0].shape[0]
    psi, q_profile = element_blocks[0][:, 0], element_blocks[0][:, 1]
    idx_map = get_matrix_map(m, kind, n_blocks, mband_k=mband_k, path=path)
    
    mats = np.zeros((n_psi, m, m), dtype=np.complex128)
    for k, (i, j) in idx_map.items():
        if k > n_blocks: continue
        mats[:, i, j] = element_blocks[k - 1][:, 2] + 1j * element_blocks[k - 1][:, 3]

    if kind in ('G'):
        il, jl = np.tril_indices(m, -1)
        mats[:, jl, il] = np.conj(mats[:, il, jl])
        
    return psi, q_profile, mats

# ────────────────────────────────────────
# 3. dedup, make_interp, plot_q_profile (이전과 동일)
# ────────────────────────────────────────
def dedup(psi, q, stacks):
    uniq_psi, idx = np.unique(psi, return_index=True)
    return uniq_psi, q[idx], {k: v[idx] for k, v in stacks.items()}

def make_interp(psi, mat_stack):
    cs_r = CubicSpline(psi, mat_stack.real, axis=0)
    cs_i = CubicSpline(psi, mat_stack.imag, axis=0)
    return lambda x, cr=cs_r, ci=cs_i: cr(x) + 1j * ci(x)

def plot_q_profile(psi, q, q_interpolant):
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(psi, q, 'o', label='Original Data Points', markersize=5)
    psi_fine = np.linspace(psi.min(), psi.max(), 300)
    ax.plot(psi_fine, q_interpolant(psi_fine), '-', label='Cubic Spline Interpolation')
    ax.set(title='Safety Factor (q) Profile', xlabel='Normalized Poloidal Flux (ψ)', ylabel='Safety Factor (q)')
    ax.legend(); ax.grid(True, linestyle='--', alpha=0.6)
    plt.show()

# ────────────────────────────────────────
# *** REVISED *** 4. 최종 행렬의 히트맵을 그리는 함수
# ────────────────────────────────────────
def plot_final_heatmaps(psi_coords, M, q_interp, F_bar_interp, G_interp, K_bar_interp, nsamp=4):
    """
    보간 함수들을 받아, 플롯에 필요한 psi 샘플에 대해서만 최종 행렬을 계산하고 히트맵을 그립니다.
    """
    names = ['F', 'G', 'K']
    samp = np.quantile(psi_coords, np.linspace(.05, .95, nsamp))
    
    fig, ax = plt.subplots(len(names), nsamp,
                        figsize=(4.5 * nsamp, 4 * len(names)),
                        constrained_layout=True, squeeze=False)
    
    I = np.identity(M.shape[0])

    for c, p in enumerate(samp):
        # --- 계산은 이 루프 안에서 즉석으로 수행됩니다 ---
        q_val = q_interp(p)
        Q_mat = M - q_val * I
        
        # 각 psi 샘플(p)에 대해 최종 행렬 계산
        final_mats = {
            'F': Q_mat @ F_bar_interp(p) @ Q_mat,
            'G': G_interp(p),
            'K': Q_mat @ K_bar_interp(p)
        }
        # ---------------------------------------------
        
        for r, n in enumerate(names):
            mat = final_mats[n]
            im = ax[r, c].imshow(np.abs(mat), cmap='viridis', origin='upper', aspect='equal')
            ax[r, c].set_title(f'{n}, ψ≈{p:.3f}')
            if r == len(names) - 1: ax[r, c].set_xlabel('j')
            if c == 0: ax[r, c].set_ylabel('i')
            plt.colorbar(im, ax=ax[r, c], label=f'|{n}|')
            
    fig.suptitle('Final F, G, K Heatmaps (Calculated On-the-fly)', y=1.03)
    plt.show()

# ────────────────────────────────────────
# 5. 메인 스크립트
# ────────────────────────────────────────

In [1]:
println("Starting matrix reading and processing...")

Starting matrix reading and processing...


In [ ]:
using LinearAlgebra
using Plots
using Interpolations
using Printf

# Global parameters
m_high = 1
m_low = -4
mdim = m_high - m_low + 1
ndim = 1
matrix_size = mdim * ndim  # 'size' 변수명 변경

"""
DCON matrix binary file reader and analyzer for Julia
- Reads F_bar, K_bar, G matrices and q profile from binary files
- Provides interpolation and visualization capabilities
- Calculates final F, K matrices on-demand for performance
"""

# ─────────────────────────────────────────────────────────────
# 1. Matrix mapping functions
# ─────────────────────────────────────────────────────────────

"""
    get_matrix_map(m::Int, kind::String, n_blocks::Int; mband_k=nothing, path="")

Determine matrix element mapping for band storage format
"""
function get_matrix_map(m::Int, kind::String, n_blocks::Int; mband_k=nothing, path="")
    if kind in ["F", "G"]
        for mb_test in 0:(m-1)
            count = sum(length(j:min(m, j + mb_test)) for j in 1:m)
            if count == n_blocks
                mband = mb_test
                println("[$path] mband = $mband (estimated for $kind)")
                
                # Create index mapping (1-based indexing)
                d = Dict{Int, Tuple{Int,Int}}()
                k = 1
                for j in 1:m
                    for i in j:min(m, j + mband)
                        d[k] = (i, j)
                        k += 1
                    end
                end
                return d
            end
        end
        error("Cannot estimate mband for $kind matrix (m=$m, n_blocks=$n_blocks)")
        
    elseif kind == "K"
        if mband_k === nothing
            error("mband_k must be specified for K matrix")
        end
        
        println("[$path] mband = $mband_k (specified value for K)")
        
        valid_element_count = sum(length(max(1, j-mband_k):min(m, j+mband_k)) for j in 1:m)
        if n_blocks > valid_element_count
            println("  Warning: File blocks ($n_blocks) > valid elements ($valid_element_count)")
        end
        
        d = Dict{Int, Tuple{Int,Int}}()
        k = 1
        for j in 1:m
            for i in max(1, j - mband_k):min(m, j + mband_k)
                d[k] = (i, j)
                k += 1
            end
        end
        return d
    else
        error("kind must be 'F', 'G', or 'K'")
    end
end

# ─────────────────────────────────────────────────────────────
# 2. Binary file reading functions
# ─────────────────────────────────────────────────────────────

"""
    read_bin(path::String, m::Int, kind::String; mband_k=nothing)

Read DCON matrix binary files with Fortran unformatted structure
"""
function read_bin(path::String, m::Int, kind::String; mband_k=nothing)
    println("📖 Reading $kind matrix from: $(basename(path))")
    
    all_records = []
    
    try
        open(path, "r") do f
            while !eof(f)
                # Read Fortran record header (4 bytes)
                if eof(f) break end
                header_bytes = read(f, 4)
                if length(header_bytes) < 4 break end
                
                record_len = reinterpret(Int32, header_bytes)[1]
                
                if record_len > 0
                    # Read data
                    data_bytes = read(f, record_len)
                    if length(data_bytes) == record_len
                        # Convert to Float32 array
                        data = reinterpret(Float32, data_bytes)
                        push!(all_records, data)
                    else
                        push!(all_records, nothing)
                    end
                else
                    push!(all_records, nothing)
                end
                
                # Read Fortran record footer (4 bytes)
                if !eof(f)
                    read(f, 4)
                end
            end
        end
    catch e
        println("❌ Error reading file: $e")
        return nothing, nothing, nothing
    end
    
    # Process records into blocks
    element_blocks = []
    current_block = []
    
    for rec in all_records
        if rec !== nothing
            push!(current_block, rec)
        else
            if !isempty(current_block)
                # 🔧 FIX: deprecated .' 연산자를 transpose()로 변경
                push!(element_blocks, transpose(hcat(current_block...)))
                current_block = []
            end
        end
    end
    
    # 🔧 FIX: 마지막 블록 처리
    if !isempty(current_block)
        push!(element_blocks, transpose(hcat(current_block...)))
    end
    
    if isempty(element_blocks)
        error("No data blocks found in file: $path")
    end
    
    n_blocks = length(element_blocks)
    n_psi = size(element_blocks[1], 1)
    
    println("  Found $n_blocks data blocks with $n_psi psi points each")
    
    # Extract psi and q profile from first block
    psi = element_blocks[1][:, 1]
    q_profile = element_blocks[1][:, 2]
    
    # Get matrix mapping
    idx_map = get_matrix_map(m, kind, n_blocks; mband_k=mband_k, path=basename(path))
    
    # Initialize matrix array
    mats = zeros(ComplexF64, n_psi, m, m)
    
    # Fill matrix elements
    for (k, (i, j)) in idx_map
        if k <= n_blocks && k <= length(element_blocks)
            # FIX: 안전한 배열 인덱싱
            block = element_blocks[k]
            if size(block, 2) >= 4
                real_part = block[:, 3]
                imag_part = block[:, 4]
                mats[:, i, j] = complex.(real_part, imag_part)
            end
        end
    end
    
    # Apply symmetry for Hermitian matrices
    if kind == "G"
        println("  Applying Hermitian symmetry")
        for ipsi in 1:n_psi
            for i in 1:m
                for j in 1:(i-1)
                    mats[ipsi, j, i] = conj(mats[ipsi, i, j])
                end
            end
        end
    end
    
    println("✅ Successfully read $kind matrix ($(size(mats)))")
    
    return psi, q_profile, mats
end

# ─────────────────────────────────────────────────────────────
# 3. Data processing functions
# ─────────────────────────────────────────────────────────────

"""
    dedup(psi, q, stacks)

Remove duplicate psi values and corresponding data
"""
function dedup(psi, q, stacks)
    # Find unique psi values
    unique_psi = unique(psi)
    unique_indices = [findfirst(==(p), psi) for p in unique_psi]
    
    deduped_q = q[unique_indices]
    deduped_stacks = Dict(k => v[unique_indices, :, :] for (k, v) in stacks)
    
    return unique_psi, deduped_q, deduped_stacks
end

"""
    make_interp(psi, mat_stack)

Create interpolation functions for complex matrix data
"""
function make_interp(psi, mat_stack)
    # Separate real and imaginary parts
    real_parts = real.(mat_stack)
    imag_parts = imag.(mat_stack)
    
    # Create interpolation objects for each matrix element
    m1, m2 = size(mat_stack)[2:3]
    
    interp_real = [linear_interpolation(psi, real_parts[:, i, j]) for i in 1:m1, j in 1:m2]
    interp_imag = [linear_interpolation(psi, imag_parts[:, i, j]) for i in 1:m1, j in 1:m2]
    
    # Return function that evaluates at given psi
    function interpolated_matrix(psi_val)
        result = zeros(ComplexF64, m1, m2)
        for i in 1:m1, j in 1:m2
            real_val = interp_real[i, j](psi_val)
            imag_val = interp_imag[i, j](psi_val)
            result[i, j] = complex(real_val, imag_val)
        end
        return result
    end
    
    return interpolated_matrix
end

"""
    plot_q_profile(psi, q, q_interpolant)

Plot safety factor profile with interpolation
"""
function plot_q_profile(psi, q, q_interpolant)
    psi_fine = range(minimum(psi), maximum(psi), length=300)
    q_fine = [q_interpolant(p) for p in psi_fine]
    
    p = plot(psi, q, seriestype=:scatter, label="Original Data Points", 
             markersize=5, xlabel="Normalized Poloidal Flux (ψ)", 
             ylabel="Safety Factor (q)", title="Safety Factor (q) Profile")
    
    plot!(p, psi_fine, q_fine, label="Linear Interpolation", linewidth=2)
    plot!(p, grid=true, gridwidth=1, gridstyle=:dash, gridalpha=0.6)
    
    display(p)
    return p
end

# ─────────────────────────────────────────────────────────────
# 4. Final matrix calculation and visualization
# ─────────────────────────────────────────────────────────────

"""
    plot_final_heatmaps(psi_coords, M, q_interp, F_bar_interp, G_interp, K_bar_interp; nsamp=4)

Plot heatmaps of final F, G, K matrices calculated on-demand
"""
function plot_final_heatmaps(psi_coords, M, q_interp, F_bar_interp, G_interp, K_bar_interp; nsamp=4)
    println("🎨 Plotting final matrix heatmaps...")
    
    names = ["F", "G", "K"]
    
    # Sample psi values
    quantiles = range(0.05, 0.95, length=nsamp)
    samp = [quantile(psi_coords, q) for q in quantiles]
    
    # Create identity matrix
    I_mat = Matrix{Float64}(LinearAlgebra.I, size(M))  # 🔧 FIX: 변수명 변경
    
    # Create subplot array
    plots_array = []
    
    for (c, p) in enumerate(samp)
        println("  Computing matrices for ψ ≈ $(round(p, digits=3))")
        
        # Calculate Q matrix
        q_val = q_interp(p)
        Q_mat = M - q_val * I_mat
        
        # Calculate final matrices on-demand
        final_mats = Dict(
            "F" => Q_mat * F_bar_interp(p) * Q_mat,
            "G" => G_interp(p),
            "K" => Q_mat * K_bar_interp(p)
        )
        
        # Create heatmaps for each matrix
        for (r, name) in enumerate(names)
            mat = final_mats[name]
            
            p_heat = heatmap(abs.(mat), 
                           title="$name, ψ≈$(round(p, digits=3))",
                           color=:viridis,
                           aspect_ratio=:equal,
                           xlabel="j", ylabel="i")
            
            push!(plots_array, p_heat)
        end
    end
    
    # Combine plots
    layout = (length(names), nsamp)
    combined_plot = plot(plots_array..., layout=layout, size=(300*nsamp, 250*length(names)))
    plot!(combined_plot, plot_title="Final F, G, K Heatmaps (Calculated On-the-fly)")
    
    display(combined_plot)
    return combined_plot
end

# ─────────────────────────────────────────────────────────────
# 5. Main analysis function
# ─────────────────────────────────────────────────────────────

"""
    analyze_dcon_matrices(; m=6, mband_k=2, files=["fmat.bin", "gmat.bin", "kmat.bin"])

Main function to read and analyze DCON matrix files
"""
function analyze_dcon_matrices(; m=6, mband_k=2, files=["fmat.bin", "gmat.bin", "kmat.bin"])
    println("🔍 DCON Matrix Analysis")
    println("="^50)
    
    # Read matrix files
    data = Dict()
    kinds = ["F", "G", "K"]
    
    for (file, kind) in zip(files, kinds)
        if isfile(file)
            if kind == "K"
                psi, q, mats = read_bin(file, m, kind; mband_k=mband_k)
            else
                psi, q, mats = read_bin(file, m, kind)
            end
            
            if psi !== nothing
                data[kind] = Dict("psi" => psi, "q" => q, "matrices" => mats)
            end
        else
            println("⚠️  File $file not found")
        end
    end
    
    if isempty(data)
        println("❌ No valid data files found")
        return nothing
    end
    
    # Use first available dataset for psi coordinates
    first_key = first(keys(data))
    psi_coords = data[first_key]["psi"]
    q_profile = data[first_key]["q"]
    
    println("\n📊 Data Summary:")
    println("  Psi range: [$(minimum(psi_coords)), $(maximum(psi_coords))]")
    println("  Number of psi points: $(length(psi_coords))")
    println("  Available matrices: $(collect(keys(data)))")
    
    # Remove duplicates
    matrix_stacks = Dict(k => v["matrices"] for (k, v) in data)
    psi_unique, q_unique, stacks_unique = dedup(psi_coords, q_profile, matrix_stacks)
    
    println("  After deduplication: $(length(psi_unique)) unique psi points")
    
    # Create interpolation functions
    interpolants = Dict()
    for (k, mats) in stacks_unique
        interpolants[k] = make_interp(psi_unique, mats)
        println("  ✅ Created interpolant for $k matrix")
    end
    
    # Create q profile interpolation
    q_interp = linear_interpolation(psi_unique, q_unique)
    
    # Plot q profile
    plot_q_profile(psi_unique, q_unique, q_interp)
    
    # Create mode number matrix M
    mode_numbers = collect(m_low:m_high)
    M = diagm(mode_numbers)
    println("\n🔢 Mode number matrix M:")
    println("  Mode range: $m_low to $m_high")
    println("  Matrix size: $(size(M))")
    
    # Plot final matrices if we have the required data
    if haskey(interpolants, "F") && haskey(interpolants, "G") && haskey(interpolants, "K")
        plot_final_heatmaps(psi_unique, M, q_interp, 
                          interpolants["F"], interpolants["G"], interpolants["K"])
    else
        println("⚠️  Cannot create final plots - missing some matrix types")
        println("   Available: $(collect(keys(interpolants)))")
    end
    
    # Return results
    return Dict(
        "psi" => psi_unique,
        "q" => q_unique,
        "interpolants" => interpolants,
        "q_interp" => q_interp,
        "M" => M,
        "raw_data" => data
    )
end

"""
    test_matrix_reading()

Quick test function for matrix reading
"""
function test_matrix_reading()
    println("🧪 Testing DCON matrix reading...")
    
    # Test with common parameters
    results = analyze_dcon_matrices(m=6, mband_k=2)
    
    if results !== nothing
        println("✅ Test completed successfully!")
        return results
    else
        println("❌ Test failed - check file availability")
        return nothing
    end
end

# Export main functions
export analyze_dcon_matrices, test_matrix_reading, read_bin, plot_q_profile
export get_matrix_map, dedup, make_interp, plot_final_heatmaps

println("📦 DCON Matrix Analysis Module Loaded")
println("Usage: results = analyze_dcon_matrices() or test_matrix_reading()")

📦 DCON Matrix Analysis Module Loaded
Usage: results = analyze_dcon_matrices() or test_matrix_reading()
Usage: results = analyze_dcon_matrices() or test_matrix_reading()


In [6]:
# ═══════════════════════════════════════════════════════════════
# 📖 DCON Matrix Reader 사용방법 (Usage Guide)
# ═══════════════════════════════════════════════════════════════

println("🚀 DCON Matrix Reader 사용방법")
println("="^60)

println("\n1️⃣ 기본 사용법 (Basic Usage):")
println("   # 모든 행렬 파일 자동 분석")
println("   results = analyze_dcon_matrices()")
println("   ")
println("   # 파라미터 조정")
println("   results = analyze_dcon_matrices(m=6, mband_k=2)")

println("\n2️⃣ 단일 파일 읽기 (Single File Reading):")
println("   # F 행렬 읽기")
println("   psi, q, F_matrices = read_bin(\"fmat.bin\", 6, \"F\")")
println("   ")
println("   # K 행렬 읽기 (mband_k 필수)")
println("   psi, q, K_matrices = read_bin(\"kmat.bin\", 6, \"K\"; mband_k=2)")

println("\n3️⃣ 개별 기능 사용 (Individual Functions):")
println("   # q 프로파일 플롯")
println("   plot_q_profile(psi, q, q_interpolant)")
println("   ")
println("   # 히트맵 플롯")
println("   plot_final_heatmaps(psi, M, q_interp, F_interp, G_interp, K_interp)")

println("\n4️⃣ 커스터마이징 (Customization):")
println("   # 특정 파일들만 분석")
println("   files = [\"fmat.bin\", \"gmat.bin\"]")
println("   results = analyze_dcon_matrices(files=files)")
println("   ")
println("   # 샘플링 수 조정")
println("   plot_final_heatmaps(...; nsamp=6)")

println("\n5️⃣ 반환값 활용 (Using Results):")
println("   results = analyze_dcon_matrices()")
println("   psi = results[\"psi\"]                    # ψ 좌표")
println("   q = results[\"q\"]                        # q 프로파일")
println("   interpolants = results[\"interpolants\"]  # 보간 함수들")
println("   q_interp = results[\"q_interp\"]          # q 보간 함수")
println("   M = results[\"M\"]                        # 모드 번호 행렬")

println("\n6️⃣ 빠른 테스트 (Quick Test):")
println("   test_matrix_reading()")

println("\n📋 파일 요구사항 (File Requirements):")
println("   • fmat.bin, gmat.bin, kmat.bin")
println("   • Fortran unformatted binary format")
println("   • 현재 작업 디렉토리에 위치")

println("\n⚙️ 파라미터 설명 (Parameters):")
println("   • m: 행렬 크기 (기본값: 6)")
println("   • mband_k: K 행렬의 밴드폭 (기본값: 2)")
println("   • nsamp: 히트맵 샘플 수 (기본값: 4)")

println("\n🎯 예제 실행 (Example Execution):")
println("   # 현재 디렉토리에 바이너리 파일이 있다면:")
println("   results = test_matrix_reading()")
println("   ")
println("   # 결과 확인:")
println("   if results !== nothing")
println("       println(\"성공! 데이터 포인트 수: \", length(results[\"psi\"]))")
println("   end")

println("\n" * "="^60)
println("🏁 Ready to analyze DCON matrices!")

🚀 DCON Matrix Reader 사용방법

1️⃣ 기본 사용법 (Basic Usage):
   # 모든 행렬 파일 자동 분석
   results = analyze_dcon_matrices()
   
   # 파라미터 조정
   results = analyze_dcon_matrices(m=6, mband_k=2)

2️⃣ 단일 파일 읽기 (Single File Reading):
   # F 행렬 읽기
   psi, q, F_matrices = read_bin("fmat.bin", 6, "F")
   
   # K 행렬 읽기 (mband_k 필수)
   psi, q, K_matrices = read_bin("kmat.bin", 6, "K"; mband_k=2)

3️⃣ 개별 기능 사용 (Individual Functions):
   # q 프로파일 플롯
   plot_q_profile(psi, q, q_interpolant)
   
   # 히트맵 플롯
   plot_final_heatmaps(psi, M, q_interp, F_interp, G_interp, K_interp)

4️⃣ 커스터마이징 (Customization):
   # 특정 파일들만 분석
   files = ["fmat.bin", "gmat.bin"]
   results = analyze_dcon_matrices(files=files)
   
   # 샘플링 수 조정
   plot_final_heatmaps(...; nsamp=6)

5️⃣ 반환값 활용 (Using Results):
   results = analyze_dcon_matrices()
   psi = results["psi"]                    # ψ 좌표
   q = results["q"]                        # q 프로파일
   interpolants = results["interpolants"]  # 보간 함수들
   q_interp = results["q_interp"]       

In [8]:
# ═══════════════════════════════════════════════════════════════
# 🚀 실제 실행 예제 (Actual Execution Examples)
# ═══════════════════════════════════════════════════════════════

println("🔥 DCON Matrix Reader - 실제 실행 예제")
println("="^60)

# 1. 기본 테스트 실행
println("\n1️⃣ 기본 테스트 실행 중...")
try
    results = test_matrix_reading()
    
    if results !== nothing
        println("✅ 성공! 다음 데이터를 읽었습니다:")
        println("   - Psi 포인트 수: ", length(results["psi"]))
        println("   - 사용 가능한 행렬들: ", collect(keys(results["interpolants"])))
        println("   - Psi 범위: [", minimum(results["psi"]), ", ", maximum(results["psi"]), "]")
    else
        println("❌ 테스트 실패 - 바이너리 파일을 확인하세요")
    end
catch e
    println("❌ 오류 발생: ", e)
    println("💡 해결 방법:")
    println("   1. fmat.bin, gmat.bin, kmat.bin 파일이 현재 디렉토리에 있는지 확인")
    println("   2. 파일들이 올바른 Fortran 바이너리 형식인지 확인")
    println("   3. 파일 접근 권한 확인")
end

println("\n" * "="^60)

# 2. 개별 파일 읽기 예제
println("\n2️⃣ 개별 파일 읽기 예제:")

# F 행렬 읽기 시도
if isfile("fmat.bin")
    println("📖 F 행렬 읽기 중...")
    try
        psi_f, q_f, mats_f = read_bin("fmat.bin", 6, "F")
        if psi_f !== nothing
            println("✅ F 행렬 성공적으로 읽음 - 크기: ", size(mats_f))
        end
    catch e
        println("❌ F 행렬 읽기 실패: ", e)
    end
else
    println("⚠️  fmat.bin 파일이 없습니다")
end

# G 행렬 읽기 시도
if isfile("gmat.bin")
    println("📖 G 행렬 읽기 중...")
    try
        psi_g, q_g, mats_g = read_bin("gmat.bin", 6, "G")
        if psi_g !== nothing
            println("✅ G 행렬 성공적으로 읽음 - 크기: ", size(mats_g))
        end
    catch e
        println("❌ G 행렬 읽기 실패: ", e)
    end
else
    println("⚠️  gmat.bin 파일이 없습니다")
end

# K 행렬 읽기 시도
if isfile("kmat.bin")
    println("📖 K 행렬 읽기 중...")
    try
        psi_k, q_k, mats_k = read_bin("kmat.bin", 6, "K"; mband_k=2)
        if psi_k !== nothing
            println("✅ K 행렬 성공적으로 읽음 - 크기: ", size(mats_k))
        end
    catch e
        println("❌ K 행렬 읽기 실패: ", e)
    end
else
    println("⚠️  kmat.bin 파일이 없습니다")
end

println("\n" * "="^60)

# 3. 파라미터 조정 예제
println("\n3️⃣ 파라미터 조정 예제:")

# 다른 행렬 크기로 시도
println("📊 다른 파라미터로 분석 시도...")
try
    # 더 작은 행렬 크기로 시도
    results_small = analyze_dcon_matrices(m=4, mband_k=1)
    if results_small !== nothing
        println("✅ m=4, mband_k=1로 성공")
    end
catch e
    println("❌ 파라미터 조정 실패: ", e)
end

try
    # 더 큰 행렬 크기로 시도
    results_large = analyze_dcon_matrices(m=8, mband_k=3)
    if results_large !== nothing
        println("✅ m=8, mband_k=3으로 성공")
    end
catch e
    println("❌ 파라미터 조정 실패: ", e)
end

println("\n" * "="^60)

# 4. 결과 활용 예제
println("\n4️⃣ 결과 활용 예제:")

try
    # 전체 분석 실행
    full_results = analyze_dcon_matrices()
    
    if full_results !== nothing
        println("🎯 분석 완료! 결과 활용 예제:")
        
        # 특정 psi 값에서 q 계산
        psi_test = 0.5
        if haskey(full_results, "q_interp")
            q_at_test = full_results["q_interp"](psi_test)
            println("   ψ = $psi_test 에서 q = $q_at_test")
        end
        
        # 특정 psi에서 행렬 계산
        if haskey(full_results["interpolants"], "F")
            F_at_test = full_results["interpolants"]["F"](psi_test)
            println("   ψ = $psi_test 에서 F 행렬 크기: ", size(F_at_test))
            println("   F 행렬 최대값: ", maximum(abs.(F_at_test)))
        end
        
        # 모드 번호 정보
        if haskey(full_results, "M")
            M = full_results["M"]
            println("   모드 번호 행렬 대각선: ", diag(M))
        end
        
        println("✅ 모든 분석이 성공적으로 완료되었습니다!")
        
    else
        println("❌ 전체 분석 실패")
    end
    
catch e
    println("❌ 결과 활용 중 오류: ", e)
end

println("\n" * "="^60)

# 5. 파일 상태 체크
println("\n5️⃣ 파일 상태 체크:")
files_to_check = ["fmat.bin", "gmat.bin", "kmat.bin"]

for file in files_to_check
    if isfile(file)
        filesize = stat(file).size
        println("✅ $file: $(filesize) bytes")
    else
        println("❌ $file: 파일 없음")
    end
end

println("\n🎉 실행 완료! 위 결과를 참고하여 필요한 수정을 진행하세요.")
println("💡 팁: 바이너리 파일이 없다면 DCON 시뮬레이션을 먼저 실행하세요.")

🔥 DCON Matrix Reader - 실제 실행 예제

1️⃣ 기본 테스트 실행 중...
🧪 Testing DCON matrix reading...
🔍 DCON Matrix Analysis
📖 Reading F matrix from: fmat.bin
❌ 오류 발생: MethodError(6, (Float32[0.01 -2.0 -0.0043648053 -1.1095033 -2.5224748 -2.025719 -2.9170477 -4.127858 -4.9592175 -4.9891815 -4.8413553 -5.614108; 0.010147579 -1.9936376 -0.0044295504 -1.1063194 -2.516075 -2.0224447 -2.9107296 -4.117077 -4.9750285 -4.991341 -4.8493247 -5.551661; 0.010590226 -1.9750948 -0.004623804 -1.0970397 -2.4974034 -2.0128925 -2.8922508 -4.0854173 -5.031177 -4.999611 -4.880668 -5.3819366; 0.0113276765 -1.9458592 -0.0049476232 -1.0824081 -2.4679353 -1.9977998 -2.8629215 -4.0356345 -5.1432977 -5.013782 -4.958804 -5.173756; 0.012359484 -1.9079996 -0.0054011024 -1.063459 -2.429785 -1.9781933 -2.8246217 -3.9730186 -5.2629514 -5.01298 -5.1271715 -5.008071; 0.013685028 -1.8637543 -0.0059843743 -1.0413119 -2.385291 -1.9551959 -2.7796302 -3.9014468 -5.192476 -4.9863243 -5.4512544 -4.940047; 0.01530351 -1.8152089 -0.00669761 -1.